In [ ]:
import camelot
import pandas as pd
from glob import glob
import os
from zipfile import ZipFile
from datetime import datetime


area1='0,600,380,570'
area2='0,570,380,400'
area3='0,400,380,300'

path_="C:/Users/ASUS/mbaandin/"
ls_file=os.listdir(path_)
now = str(datetime.now())

def pdf_to_excel(file,psswd,area,namafile,path):
    tables = camelot.read_pdf(path+file,password=psswd,flavor='stream', table_areas=[area])
    dat=tables[0].df # get a pandas DataFrame!

    date = pd.to_datetime(dat[0], errors='coerce')
    ket=[]
    b=dat[1].str.split(n=1).str[0]
    b1=dat[1].str.split(n=1).str[1]
    c=pd.to_datetime(b, errors='coerce')
    for i in range(len(c)):
        if pd.isnull(c[i])==True:
            ket.append('{} {}'.format(b[i],b1[i]))
        else:
            ket.append(b1[i])

    val={
        'Tanggal Transaksi':date,
        'Tanggal Pembukuan':c,
        'Rincian Transaksi':ket,
        'Jumlah (Rp)':dat[2].replace('.', '')
    }

    billing=pd.DataFrame(val)
    if area=='0,600,380,570' or area=='0,400,380,300':
        billing['Tanggal Transaksi'][1]=dat[0][1]
    return billing

for file in ls_file:
    passwd=file.split(' - ')[1].split('.pdf')[0]
    tmp=pdf_to_excel(file,passwd,area1,file,path_).append(pdf_to_excel(file,passwd,area2,file,path_))
    try:
        tmp=tmp.append(pdf_to_excel(file,passwd,area3,file,path_))
    except:
        pass
    tmp['Rincian Transaksi']=tmp['Rincian Transaksi'].str.replace(' nan','')
    tmp.to_excel("{}{}.xlsx".format(path_,file[:-4]),index=False)

with ZipFile("CC Receipt PDF to XLSX.zip".format(now), 'w') as zipObj2:
    for excel in glob(path_+'*xlsx'):
        zipObj2.write(excel)